Notebook

# Climate Displacement

Disaster Types
- Year
- State
- Households Inflow (Number of Returns)
- Households Outflow (Number of Returns)
- Individuals Inflow (Number of Exemptions)
- Individuals Outflow (Number of Exemptions)
- Chemical
- Dam/Levee Break
- Drought
- Earthquake
- Fire
- Flood
- Human Cause
- Hurricane
- Ice
- Mud/Landslide
- Other
- Snow
- Storm
- Terrorism
- Tornado
- Tsunami
- Typhoon
- Volcano
- Water
- Winter

In [ ]:
"""
Folders Setup

code 
    notebook.ipynb
    data
        Disasters
            FEMA_dataset.csv
        StateMigration
            1990to1991StateMigration
                 1990to1991StateMigrationInflow
                     Alabama91in.xls
                     Alaska91in.xls
                     .
                     .
                     .
                     Wisconsin91in.xls
                     Wyoming91in.xls 
                 1990to1991StateMigrationOutflow
                     Alabama91Out.xls
                     Alaska91Out.xls 
                     .
                     .
                     .
                     Wisconsin91Out.xls
                     Wyoming91Out.xls 
            .
            .
            .
            2008to2009StateMigration
            2009to2010StateMigration
            2010to2011StateMigration 
"""

## Pseudocode:

### FEMA Dataset Pre-processing (Neely)
1. Create new FEMA_dataset with columns 
    - contains Year, State, Disaster Type
2. Name file "State_Disasters_by_Year"

### StateMigration Data Pre-Processing (Ben)
1. Convert all datasets in StateMigration from .xls into .csv files
2. Extract "Total Flow" row with "Number of Returns" and "Number of Exemptions" - assign I if from inflow and O if from outflow - from every state file.
3. Extract "State" and "Year" from every file
4. Create file with "State" (from file name), "Year" (from file name), "Number_of_Returns_I", "Number_of_Exemptions_I", "Number_of_Returns_I" and "Number_of_Exemptions_O"
5. Name file "State Migration by Year"

### Merge Datasets (Both)
1. Merge datasets on common attributes "Year" and "State"
2. Name dataset "State_Migration_and_Disasters_by_Year"

#### Train and Testing
1. Create training and testing datasets
    Questions: How should we split training and testing data?
2. Create Neural Network models
    Input: Year, State, Disaster Type
    Output: Migration Inflow (Household/Individual), Migration Outflow (Household/Individual)
3. Put training and testing through the Neural Network models.
4. Evaluate which models are the most effective.
---
Data Augmentation
Synthetic Data
Use svm or decision tree -- skleant -- and compare against a neural network
could use all data for training and all data for validation - not this is a faulty practice in 
20% distribution of state 


```
read_file = pd.read_excel ("Test.xlsx")
 
# Write the dataframe object
# into csv file
read_file.to_csv ("Test.csv",
                  index = None,
                  header=True)
```

In [2]:
# adding imports
import shutil
import pandas as pd
import os
import glob
import xlrd
import csv
import numpy as np
#from fastai import *

### FEMA Dataset Pre-processing

Creates State_Disasters_by_Year.csv with:
- State
- Disaster Type
- Start Year
- End Year

In [ ]:
# FEMA Dataset Preprocessing

# copy original FEMA dataset to new file
original = r'../code/data/Disasters/FEMA_dataset.csv'
new = r'../code/data/Disasters/State_Disasters_by_Year.csv'
shutil.copyfile(original, new)

# read csv file
data = pd.read_csv('~/code/data/Disasters/State_Disasters_by_Year.csv')

# delete irrelevant rows
data.pop('Declaration Number')
data.pop('Declaration Type')
data.pop('Declaration Date')
data.pop('County')
data.pop('Disaster Title')
data.pop('Close Date')
data.pop('Individual Assistance Program')
data.pop('Public Assistance Program')
data.pop('Hazard Mitigation Program')
data.pop('Individuals & Households Program')

# extract years
data['Start Year'] = pd.DatetimeIndex(data['Start Date']).year
data['End Year'] = pd.DatetimeIndex(data['End Date']).year

# delete start and end dates
data.pop('Start Date')
data.pop('End Date')

print(data)

# save changes csv
data.to_csv('../code/data/Disasters/State_Disasters_by_Year.csv')

### Converting State Migration data from .xls to .csv

Converting all datasets in StateMigration from .xls to .csv

In [ ]:
# Convert all files in StateMigration folder from .xls to .csv

# create list of xls files
xls_list = glob.glob("/Users/ben/Desktop/climate-displacement/code/data/StateMigration/*/*/*.xls")

# replace xls 
for xls_file in xls_list:
    
    wb = xlrd.open_workbook(xls_file)
    sh = wb.sheet_by_index(0)
    csv_file = open(xls_file[0:-3]+'csv', "w")
    wr = csv.writer(csv_file, quoting=csv.QUOTE_ALL)
    
    for rownum in range(sh.nrows):
        wr.writerow(sh.row_values(rownum))
        
    csv_file.close()
    
    # remove .xls files
    os.remove(xls_file)


### More data wrangling - StateMigration dataset
- Extract "Total Flow" row with "Number of Returns" and "Number of Exemptions"
- Assign "I" if from inflow and "O" if from outflow - for every state file
- Extract "State" and "Year" from every file
- Create file with "State" (from file name), "Year" (from file name), "Number of Returns_I", "Number of Exemptions I", "Number of Returns O" and "Number of Exemptions O"

### Pseudocode

1. create output path in repository for the merged StateMigration dataset

2. read each csv file in the StateMigration folder, and for each file, 
    - d
3. check the csv files to make sure they are the intended data
4. remove the original csv files

In [ ]:
# set new file location for merged StateMigration dataset
output_path = r'../code/data/StateMigration/State_Migrations_by_Year.csv'

In [ ]:
# create the output file at output_path
output = open(output_path, "w")
output.close()

In [ ]:
# create empty DataFrame object
df = pd.DataFrame()
df.insert(0,'State', '')
df.insert(1,'Year', '')
df.insert(2,'NOR(I)', '')
df.insert(3,'NOE(I)', '')
df.insert(4,'NOR(O)', '')
df.insert(5,'NOE(O)', '')

# print(df)

In [3]:
# create dictionary of "state initial keys" with multiple "values"
# run each segment through dictionary, and convert into state initial
stateDict = {
    "AL":['Alabama', 'al', 'AL', 'alab', 'Alab'],
    "AK":['Alaska', 'ak', 'AK', 'alas', 'Alas'],
    "AZ":['Arizona', 'az', 'AZ', 'ariz', 'Ariz'],
    "AR":['Arkansas', 'ar', 'AR', 'arka', 'Arka', 'aka'],
    "CA":['California', 'ca', 'CA', 'cali', 'Cali'],
    "CO":['Colorado', 'co', 'CO', 'colo', 'Colo'],
    "CT":['Connecticut', 'ct', 'CT', 'conn', 'Conn'],
    "DE":['Delaware', 'de', 'DE', 'dela', 'Dela'],
    "DC":['DistrictofColumbia', 'Districtofcolumbia', 'District of Columbia', 'dc', 'DC', 'dist', 'Dist', 'DiCo', 'dico'],
    "FL":['Florida', 'fl', 'FL', 'flor', 'Flor'],
    "GA":['Georgia', 'ga', 'GA', 'geor', 'Geor'],
    "HI":['Hawaii', 'hi', 'HI', 'hawa', 'Hawa'],
    "ID":['Idaho', 'id', 'ID', 'idah', 'Idah'],
    "IL":['Illinois', 'il', 'IL', 'illi', 'Illi'],
    "IN":['Indiana', 'in', 'IN', 'indi', 'Indi'],
    "IA":['Iowa', 'ia', 'IA', 'iowa'],
    "KS":['Kansas', 'ks', 'KS', 'kans', 'Kans'],
    "KY":['Kentucky', 'ky', 'KY', 'kent', 'Kent'],
    "LA":['Louisiana', 'la', 'LA', 'loui', 'Loui'],
    "MA":['Massachusetts', 'ma', 'MA', 'mass', 'Mass'],
    "MD":['Maryland', 'md', 'MD', 'mary', 'Mary'],
    "ME":['Maine', 'me', 'ME', 'main', 'Main'],
    "MI":['Michigan', 'mi', 'MI', 'mich', 'Mich'],
    "MN":['Minnesota', 'mn', 'MN', 'minn', 'Minn'],
    "MO":['Missouri', 'mo', 'MO', 'Miso', 'miso'],
    "MS":['Mississippi', 'ms', 'MS', 'Misi', 'misi', 'miss', 'Miss'],
    "MT":['Montana', 'mt', 'MT', 'mont', 'Mont'],
    "NC":['North Carolina', 'NorthCarolina', 'nc', 'NC', 'NoCa', 'noca', 'ncar', 'Northcarolina'],
    "ND":['North Dakota', 'NorthDakota', 'nd', 'ND', 'NoDa', 'noda', 'ndak', 'Northdakota'],
    "NE":['Nebraska', 'ne', 'NE', 'Nebr', 'nrbt', 'nebr'],
    "NH":['New Hampshire', 'NewHampshire', 'nh', 'NH', 'NeHa', 'neha', 'newh'],
    "NJ":['New Jersey', 'NewJersey', 'nj', 'NJ', 'NeJe', 'neje', 'newj', 'Newjersey'],
    "NM":['New Mexico', 'NewMexico', 'nm', 'NM', 'NeMe', 'neme', 'newm', 'Newmexico'],
    "NV":['Nevada', 'nv', 'NV', 'Neva', 'neva'],
    "NY":['New York', 'NewYork', 'ny', 'NY', 'newy', 'NeYo', 'neyo', 'newY','Newyork'],
    "OH":['Ohio', 'oh', 'OH', 'ohio', 'nhio'],
    "OK":['Oklahoma', 'ok', 'OK', 'okla', 'Okla'],
    "OR":['Oregon', 'or', 'OR', 'oreg', 'Oreg', 'oeg'],
    "PA":['Pennsylvania', 'pa', 'PA', 'penn', 'Penn'],
    "RI":['Rhode Island', 'RhodeIsland', 'ri', 'RI', 'Rhls', 'rhod', 'Rhod', 'RhIs'],
    "SC":['South Carolina', 'SouthCarolina', 'sc', 'SC', 'SoCa', 'soca', 'scar', 'Southcarolina'],
    "SD":['South Dakota', 'SouthDakota', 'sd', 'SD', 'SoDa', 'soda', 'sdak', 'Southdakota'],
    "TN":['Tennessee', 'tn', 'TN', 'Tenn', 'tenn'],
    "TX":['Texas', 'tx', 'TX', 'texa', 'Texa'],
    "UT":['Utah', 'ut', 'UT', 'utah'],
    "VA":['Virginia', 'va', 'VA', 'virg', 'Virg', 'vrg'],
    "VT":['Vermont', 'vt', 'VT', 'verm', 'Verm'],
    "WA":['Washington', 'wa', 'WA', 'wash', 'Wash'],
    "WI":['Wisconsin', 'wi', 'WI', 'wisc', 'Wisc', 'wiso', 'wsc'],
    "WV":['West Virginia', 'WestVirginia', 'wv', 'WV', 'west', 'wevi', 'wvir', 'Westvirginia'],
    "WY":['Wyoming', 'wy', 'WY', 'wyom', 'Wyom']    
}

def getKey(val):
    for key, valueList in stateDict.items():
         for value in valueList:
            if val == value:
                 return key
    
    return False

print(getKey('Wisconsin'))

WI


In [4]:
# read csv file
csv_list = glob.glob("/Users/ben/Desktop/climate-displacement/code/data/StateMigration/*/*/*.csv")

# create list of row_params
list_param = []

for csv_file in csv_list:
    # os.path.split returns a list of (head, tail) where head is the parent directories 
    # and tail is the filename and extension
    temp = os.path.split(csv_file)
    temp2 = os.path.split(temp[0])
    
    # get file name and parent folder from temp, temp2 respectively
    filename = temp[1]
    parentfile = temp2[1]
    
    # print (filename, parentfile)
    # print (type(filename))
    
    # extract state, year, and inflow/outflow
    # three different naming conventions in the StateMigration dataset
    # 1) [State][Year1Year2 e.g. (0708)][in/out]
    # 2) [State][Year2 e.g. 91][In/Out]
    # 3) [Year1Year2 like 1)]inmig[in/out][state INITIAL e.g. AL]
    # 4) [first 4 letters of State][Year2][in/ot]
    # 5) s9[last digits of Year1, Year2 e.g. 56][state INITIAL][ir/or]
    # 6) same as 4) but with extra "r" at the end
    
    # naming convention 1: used for years 2004-2009
    name1 = [2004,2005,2006,2007,2008]
    # naming convention 2: used for years 1990-1993
    name2 = [1990,1991,1992]
    # naming convention 3: used for years 2009-2011
    name3 = [2009,2010]
    # naming convention 4: used for years 1993-1995, 1996-2000, 2001-2004
    name4 = [1993,1994,1996,1997, 1998, 1999, 2001,2002,2003]
    # naming convention 5: used for years 1995-1996
    name5 = 1995
    # naming convention 6: used for years 2000-2001
    name6 = 2000
    
    # extract inflow/outflow, year using parentfile, and state using filename
    if parentfile[-6] == 'u':
        io = parentfile[-7:]
    elif parentfile[-5] == 'n':
        io = parentfile[-6:]
    year = int(parentfile[0:4])
    
    #2009to2010StateMigrationInflow
    # print(io, year)
    
    if year in name1:
        if io == 'Inflow':
            state = filename[:-10]
        elif io == 'Outflow':
            state = filename[:-11]
    elif year in name2:
        if io == 'Inflow':
            state = filename[:-8]
        elif io == 'Outflow':
            state = filename[:-9]
    elif year in name3:
        state = filename[-6:-4]
    elif year in name4:
        state = filename[:4]
        if state == 'vrg9':
            state = 'vrg'
        elif state == 'vrg0':
            state = 'vrg'
        elif state == 'az94':
            state = 'az'
        elif state == 'aka9':
            state = 'aka'
        elif state == 'wsc9':
            state = 'wsc'
    elif year == name5:
        state = filename[-8:-6]
    elif year == name6:
        state = filename[:4]
        if state == 'vrg0':
            state = 'vrg'
        elif state == 'oeg0':
            state = 'oeg'

    si = getKey(state)
    if si != False:
        row_param = [si, year, io]
    # print(row_param)
    
    # the total flow data in each years are located in different rows and columns.
    # type1 - 1990, 1991: located in row 9, columns D and F
    type1 = [1990, 1991]
    # type2 - 1992-1994, 2004-2006: located in row 9, columns D and E
    type2 = [1992,1993,1994,2004,2005,2006]
    # type3 - 1995-2003, 2007-2008: located in row 10, columns D and E
    type3 = [1995,1996,1997,1998,1999,2000,2001,2002,2003,2007,2008]
    # type4 - 2009-2010: located in row 8, columns E and F
    type4 = [2009,2010]
    
    data = pd.read_csv(csv_file)
    # print(data)
    if si != False:
        if year in type1:
            # total = data.iloc[7] #7, np.r_[3,5]
            totaltemp = data.iat[7,0]
            list = totaltemp.split(",")
            nor = list[3]
            noe = list[5]
        elif year in type2:
            # total = data.iloc[7] #7
            totaltemp = data.iat[7,0]
            #print(totaltemp)
            list = totaltemp.split(",")
            # print(list)
            nor = list[3]
            noe = list[4]
        elif year in type3:
            if year == 2003:
                nor = data.iat[8,4]
                noe = data.iat[8,5]
            elif year == 1997:
                nor = data.iat[8,4]
                noe = data.iat[8,5]
            else:
                totaltemp = data.iat[7,0]
                list = totaltemp.split(",")
                nor = list[3]
                noe = list[4]
        elif year in type4:
            total = data.iloc[6]
            nor = data.iat[6,4]
            noe = data.iat[6,5]
        # print(data)    
        # print(total)
        # print(nor, noe)
        # print(data)
        
        row_param.append(nor)
        row_param.append(noe)
        
        list_param.append(row_param)
        # print(row_param)
        # data.shape
        # break
    
    

In [ ]:
df.columns
print(df)
df.columns.values.tolist() # headers

test1 = ["FL", 1999, 14683, 19699, 12666, 18671]
test_df = pd.DataFrame(columns=['State', 'Year', 'NOR(I)', 'NOE(I)', 'NOR(O)', 'NOE(O)'])
# test1 = pd.Series(test1, index=["FL", 1999, 14683, 19699, 12666, 18671])
#test_df.append([test1])

# New list for append into df
list = ["Saurabh", 23, "Delhi", "india"]
 
# using loc methods
df.loc[len(df)] = list
 
# display
display(df)

In [5]:
# create a current list of what has been added to the list
current = []
# add parameters to df object
for row in list_param:
    # nested for loop to compare state and year of current list to list in question
    if len(current) > 0: 
        for row2 in current:
            # if statement comparing state and year (indices 0 and 1 respectively)
            if (row2[0] == row[0]) and (row2[1] == row[1]):
                # check if row has inflow or outflow data looking at element at index 2
                if row2[2] == "I":
                    # then add to inflow columns located at indices 2 and 3 of *row2* in current
                    row2.insert(2,row[3])
                    row2.insert(3,row[4])
                    row2[2] == "IO"
                elif row[2] == "O":
                    # then add to outflow columns at the end of the row2 in current
                    row2.append(row[3])
                    row2.append(row[4])
                    row2[2] == "IO"
            else:
                # create new list object and append to current
                if (row[2] == "Inflow"):
                    newrow = [row[0],row[1],"I",row[3],row[4]]
                elif (row[2] == "Outflow"):
                    newrow = [row[0],row[1],"O",row[3],row[4]]
                current.append(newrow)
    else:
        # create new list object and append to current
        if (row[2] == "Inflow"):
            newrow = [row[0],row[1],"I",row[3],row[4]]
        elif (row[2] == "Outflow"):
            newrow = [row[0],row[1],"O",row[3],row[4]]
        current.append(newrow) 

KeyboardInterrupt: 

In [ ]:
# create a current list of what has been added to the list
current = []
# add parameters to df object
for row in list_param:
    # nested for loop to compare state and year of current list to list in question
    if len(current) > 0: 
        for row2 in current:
            # if statement comparing state and year (indices 0 and 1 respectively)
            if (row2[0] == row[0]) and (row2[1] == row[1]):
                # check if row has inflow or outflow data looking at element at index 2
                if row2[2] == "I":
                    # then add to inflow columns located at indices 2 and 3 of *row2* in current
                    row2.insert(2,row[3])
                    row2.insert(3,row[4])
                    row2[2] == "IO"
                elif row[2] == "O":
                    # then add to outflow columns at the end of the row2 in current
                    row2.append(row[3])
                    row2.append(row[4])
                    row2[2] == "IO"
            else:
                # create new list object and append to current
                if (row[2] == "Inflow"):
                    newrow = [row[0],row[1],"I",row[3],row[4]]
                elif (row[2] == "Outflow"):
                    newrow = [row[0],row[1],"O",row[3],row[4]]
                current.append(newrow)
    else:
        # create new list object and append to current
        if (row[2] == "Inflow"):
            newrow = [row[0],row[1],"I",row[3],row[4]]
        elif (row[2] == "Outflow"):
            newrow = [row[0],row[1],"O",row[3],row[4]]
        current.append(newrow) 

In [ ]:
print(current)

In [ ]:
# parse out the row from csv file for NOR, NOE

In [ ]:
# convert df into csv

In [ ]:
# close output csv file
output.close()

In [ ]:
"""
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f) and filename.endswith('.txt'):
        read_file = pd.read_excel (r'../code/data/StateMigration/1990to1991StateMigration/1990to1991StateMigrationInflow/Alabama91in.xls')
    read_file.to_csv (r'../code/data/StateMigration/1990to1991StateMigration/1990to1991StateMigrationInflow/Alabama91in.csv', index = None, header=True)


read_file = pd.read_excel (r'../code/data/StateMigration/1990to1991StateMigration/1990to1991StateMigrationInflow/Alabama91in.xls')
read_file.to_csv (r'../code/data/StateMigration/1990to1991StateMigration/1990to1991StateMigrationInflow/Alabama91in.csv', index = None, header=True)



location = "/Users/neely/Desktop/climate-displacement/code/data/StateMigration/1990to1991StateMigration/1990to1991StateMigrationInflow"
placement = "./Users/neely/Desktop/climate-displacement/code/data/StateMigration"

for file in glob.glob("*.xls"):
"""

"""
from os.path import isfile, join
onlyfiles = [f for f in listdir('../code/data/StateMigration/') if isfile(join('../code/data/StateMigration/', f))]
"""

In [ ]:
# Neural Network -> fastai

# path to dataset here (./Users/neely/Desktop/climate-displacement/code/data/StateMigration)
path = untar_data(URLs.MNIST)
path.ls()

# data loader
dls = ImageDataLoaders.from_folder(path, train="training")